In [ ]:
import nbformat as nbf
from textwrap import dedent
from pathlib import Path

In [ ]:
nb = nbf.v4.new_notebook()
cells = []

In [ ]:
cells.append(nbf.v4.new_markdown_cell(dedent("""
# Analítica rápida de OK/NOK por Fecha y Hora
Este cuaderno lee un archivo **Excel** con campos de **estado (OK/NOK)**, **fecha** y **hora**, y produce:
- Un **gráfico de dispersión** (Fecha vs Hora) separando OK y NOK.
- Un **gráfico de barras** con **conteo de NOK por hora**.
- Un **informe PDF** con ambos gráficos y un resumen descriptivo.

> Cambia la ruta del archivo en la siguiente celda de parámetros y ejecuta todo.
""")))

In [ ]:
cells.append(nbf.v4.new_code_cell(dedent("""
# ==========================
# Parámetros
# ==========================
excel_path = "ejecuciones.xlsx"   # <- Cambia a tu ruta/archivo .xlsx
sheet_name = None                 # Ej: "Hoja1" o None para primera hoja

# Salidas
out_dir = "salida_ok_nok"
scatter_png = f"{out_dir}/scatter_fecha_hora.png"
bars_png = f"{out_dir}/barras_nok_por_hora.png"
pdf_path = f"{out_dir}/informe_ok_nok.pdf"
""")))

In [ ]:
cells.append(nbf.v4.new_code_cell(dedent("""
# ==========================
# Imports y utilidades
# ==========================
# Si falta alguna librería, instala con:
# !pip install pandas matplotlib openpyxl reportlab

import os, re, math
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader

SPANISH_MONTHS = {
    "enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6,
    "julio": 7, "agosto": 8, "septiembre": 9, "setiembre": 9, "octubre": 10,
    "noviembre": 11, "diciembre": 12
}

def normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [re.sub(r"\\s+", " ", str(c)).strip().lower() for c in df.columns]
    return df

def guess_col(df: pd.DataFrame, candidates):
    for cand in candidates:
        for c in df.columns:
            if cand in c:
                return c
    return None

def parse_spanish_date(val):
    if pd.isna(val):
        return pd.NaT
    if isinstance(val, (pd.Timestamp, datetime)):
        return pd.Timestamp(val).normalize()
    if isinstance(val, (int, float)) and not math.isnan(val):
        try:
            return pd.to_datetime('1899-12-30') + pd.to_timedelta(int(val), unit='D')
        except Exception:
            pass
    s = str(val).strip().lower()
    t = pd.to_datetime(s, dayfirst=True, errors='coerce')
    if not pd.isna(t):
        return pd.Timestamp(t).normalize()
    m = re.search(r'(\\d{1,2})\\s+de\\s+([a-záéíóúñ]+)\\s+de\\s+(\\d{4})', s)
    if m:
        dd = int(m.group(1))
        mes_txt = (m.group(2)
                   .replace('á','a').replace('é','e').replace('í','i')
                   .replace('ó','o').replace('ú','u'))
        mm = SPANISH_MONTHS.get(mes_txt, None)
        yyyy = int(m.group(3))
        if mm:
            return pd.Timestamp(year=yyyy, month=mm, day=dd)
    return pd.NaT

def parse_time(val):
    if pd.isna(val):
        return pd.NaT
    if isinstance(val, (pd.Timestamp, datetime)):
        return pd.Timestamp('1970-01-01') + pd.to_timedelta(val.hour, 'h') + \
               pd.to_timedelta(val.minute, 'm') + pd.to_timedelta(val.second, 's')
    s = str(val).strip().lower()
    s = (s.replace('a. m.', 'am').replace('p. m.', 'pm')
           .replace('a. m', 'am').replace('p. m', 'pm')
           .replace(' a m', 'am').replace(' p m', 'pm')
           .replace('a.m.', 'am').replace('p.m.', 'pm')
           .replace(' a.m', 'am').replace(' p.m', 'pm'))
    for fmt in ['%I:%M:%S %p','%H:%M:%S','%I:%M %p','%H:%M']:
        t = pd.to_datetime(s, format=fmt, errors='coerce')
        if not pd.isna(t):
            return pd.Timestamp('1970-01-01') + pd.to_timedelta(int(t.dt.hour.iloc[0]), 'h') + \
                   pd.to_timedelta(int(t.dt.minute.iloc[0]), 'm') + \
                   pd.to_timedelta(int(getattr(t.dt, 'second', pd.Series([0])).iloc[0]), 's')
    t = pd.to_datetime(s, errors='coerce')
    if not pd.isna(t):
        return pd.Timestamp('1970-01-01') + pd.to_timedelta(int(t.dt.hour.iloc[0]), 'h') + \
               pd.to_timedelta(int(t.dt.minute.iloc[0]), 'm') + \
               pd.to_timedelta(int(getattr(t.dt, 'second', pd.Series([0])).iloc[0]), 's')
    return pd.NaT

Path(out_dir).mkdir(parents=True, exist_ok=True)
""")))

In [ ]:
cells.append(nbf.v4.new_code_cell(dedent("""
# ==========================
# Cargar datos
# ==========================
df = pd.read_excel(excel_path, sheet_name=sheet_name)
df = normalize_cols(df)

col_status = guess_col(df, ['ok/nok', 'oknok', 'ok nok', 'ejecucion ok/nok', 'estado', 'resultado'])
col_fecha  = guess_col(df, ['fecha'])
col_hora   = guess_col(df, ['hora', 'hora ejecu'])

if not col_status or not col_fecha or not col_hora:
    raise ValueError(f"No pude identificar columnas. Detectadas -> status:{col_status}, fecha:{col_fecha}, hora:{col_hora}")

df['_status'] = df[col_status].astype(str).str.strip().str.lower()
df['_status'] = np.where(df['_status'].str.contains('nok'), 'NOK', 'OK')
df['_fecha'] = df[col_fecha].apply(parse_spanish_date)
df['_hora']  = df[col_hora].apply(parse_time)

df = df.dropna(subset=['_fecha', '_hora']).copy()
df['_hour'] = df['_hora'].dt.hour

print("Filas válidas:", len(df))
df.head(5)
""")))


In [ ]:
cells.append(nbf.v4.new_markdown_cell("## Gráfico 1: Dispersión **Fecha vs Hora** (OK vs NOK)"))
cells.append(nbf.v4.new_code_cell(dedent("""
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

fig, ax = plt.subplots(figsize=(10,5))
for label in ['NOK', 'OK']:
    sub = df[df['_status']==label]
    ax.scatter(sub['_fecha'], sub['_hora'], label=label, alpha=0.9)

ax.set_xlabel('Fecha')
ax.set_ylabel('Hora del día')
ax.xaxis.set_major_locator(mdates.AutoDateLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax.yaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.grid(True, linestyle='--', linewidth=0.5, alpha=0.6)
ax.legend(title='Estado')
fig.autofmt_xdate()
plt.tight_layout()
plt.savefig(scatter_png, dpi=150)
plt.show()
print(f"Guardado: {scatter_png}")
""")))

In [ ]:
cells.append(nbf.v4.new_markdown_cell("## Gráfico 2: Barras — **Conteo de NOK por hora (0–23)**"))
cells.append(nbf.v4.new_code_cell(dedent("""
nok = df[df['_status']=='NOK'].copy()
conteo_hora = nok.groupby('_hour').size().reindex(range(24), fill_value=0)

fig, ax = plt.subplots(figsize=(10,4))
ax.bar(conteo_hora.index.astype(str), conteo_hora.values)
ax.set_xlabel('Hora del día')
ax.set_ylabel('NOK (conteo)')
ax.grid(True, axis='y', linestyle='--', linewidth=0.5, alpha=0.6)
plt.tight_layout()
plt.savefig(bars_png, dpi=150)
plt.show()
print(f"Guardado: {bars_png}")
""")))

In [ ]:
cells.append(nbf.v4.new_markdown_cell("## Resumen descriptivo"))
cells.append(nbf.v4.new_code_cell(dedent("""
total = len(df)
tot_ok = (df['_status']=='OK').sum()
tot_nok = (df['_status']=='NOK').sum()
top_horas = conteo_hora.sort_values(ascending=False).head(5)

print(f"Total registros: {total}")
print(f"OK: {tot_ok} | NOK: {tot_nok}")
print("\\nTop 5 horas con más NOK:")
for h, v in top_horas.items():
    print(f"  - {h:02d}:00  ->  {v} NOK")

In [ ]:
# Texto para PDF
resumen = [
    f"Total registros: {total}",
    f"OK: {tot_ok} | NOK: {tot_nok}",
    "",
    "Top 5 horas con más NOK:"
] + [f"  - {int(h):02d}:00 -> {int(v)} NOK" for h, v in top_horas.items()]
""")))

cells.append(nbf.v4.new_markdown_cell("## Generar **PDF** con gráficos y resumen"))
cells.append(nbf.v4.new_code_cell(dedent("""
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm
from reportlab.lib.utils import ImageReader
from pathlib import Path

Path(out_dir).mkdir(parents=True, exist_ok=True)

c = canvas.Canvas(pdf_path, pagesize=A4)
W, H = A4

# Portada
c.setFont("Helvetica-Bold", 16)
c.drawString(2*cm, H-3*cm, "Informe OK/NOK — Fecha y Hora")
c.setFont("Helvetica", 10)
c.drawString(2*cm, H-4*cm, f"Archivo: {excel_path}")
c.drawString(2*cm, H-4.6*cm, f"Hoja: {sheet_name if sheet_name else '(primera)'}")
c.drawString(2*cm, H-5.2*cm, f"Total: {total}  |  OK: {tot_ok}  |  NOK: {tot_nok}")
c.showPage()

In [ ]:
# Resumen
c.setFont("Helvetica-Bold", 12)
c.drawString(2*cm, H-2.5*cm, "Resumen")
c.setFont("Helvetica", 10)
y = H-3.2*cm
for line in resumen:
    c.drawString(2*cm, y, line)
    y -= 0.6*cm
    if y < 3*cm:
        c.showPage()
        c.setFont("Helvetica", 10)
        y = H-2.5*cm


In [ ]:
# Gráfico de dispersión
if Path(scatter_png).exists():
    c.showPage()
    c.setFont("Helvetica-Bold", 12)
    c.drawString(2*cm, H-2.5*cm, "Dispersión: Fecha vs Hora (OK/NOK)")
    img = ImageReader(scatter_png)
    img_w = W - 3*cm
    img_h = (img_w * 9) / 16
    c.drawImage(img, 1.5*cm, H-2.5*cm-img_h-0.5*cm, width=img_w, height=img_h, preserveAspectRatio=True)


In [ ]:
# Barras NOK por hora
if Path(bars_png).exists():
    c.showPage()
    c.setFont("Helvetica-Bold", 12)
    c.drawString(2*cm, H-2.5*cm, "Barras: NOK por hora")
    img = ImageReader(bars_png)
    img_w = W - 3*cm
    img_h = (img_w * 9) / 16
    c.drawImage(img, 1.5*cm, H-2.5*cm-img_h-0.5*cm, width=img_w, height=img_h, preserveAspectRatio=True)
    c.save()
print(f"PDF generado en: {pdf_path}")
""")))

nb_path = Path("/mnt/data/analitica_ok_nok_simple.ipynb")
nb_path.write_text(nbf.writes(nb), encoding="utf-8")

print(nb_path.as_posix())
